Converting table of PFF team grades in each category to rankings (1-32) to be used as a feature in PBP Model 

In [160]:
import pandas as pd

In [161]:
pff_grades_2018 = pd.read_excel('../../Data/pff-data/nfl-ratings-2018.xlsx')
pff_grades_2019 = pd.read_excel('../../Data/pff-data/nfl-ratings-2019.xlsx')
pff_grades_2020 = pd.read_excel('../../Data/pff-data/nfl-ratings-2020.xlsx')

In [162]:
pff_grades_2020 = pff_grades_2020.rename(columns={'TEAM' : 'Team'})
pff_grades_2020_def = pff_grades_2020[['Team', 'DEF']]
pff_grades_2020_def = pff_grades_2020_def.sort_values(by='DEF', ascending=False, ignore_index=True) #sort by defense grade
pff_grades_2020_def['season'] = 2021
pff_grades_2020_def['Previous_DEF_rank'] = [i + 1 for i in range(len(pff_grades_2020_def))] #ranking is now the index+1 in this df

In [163]:
pff_grades_2019_def = pff_grades_2019[['Team', 'DEF']]
pff_grades_2019_def = pff_grades_2019_def.sort_values(by='DEF', ascending=False, ignore_index=True) #sort by defense grade
pff_grades_2019_def['season'] = 2020
pff_grades_2019_def['Previous_DEF_rank'] = [i + 1 for i in range(len(pff_grades_2019_def))] #ranking is now the index+1 in this df

In [164]:
pff_grades_2018_def = pff_grades_2018[['Team', 'DEF']]
pff_grades_2018_def = pff_grades_2018_def.sort_values(by='DEF', ascending=False, ignore_index=True)
pff_grades_2018_def['season'] = 2019
pff_grades_2018_def['Previous_DEF_rank'] = [i + 1 for i in range(len(pff_grades_2018_def))]

In [165]:
pff_grades_2018_def = pff_grades_2018_def.drop(columns=['DEF'])
pff_grades_2019_def = pff_grades_2019_def.drop(columns=['DEF'])
pff_grades_2020_def = pff_grades_2020_def.drop(columns=['DEF'])
defense_grades_df = pd.concat([pff_grades_2018_def, pff_grades_2019_def, pff_grades_2020_def], axis=0)

In [166]:
team_replacements = {'Pittsburgh Steelers': 'PIT', 'San Francisco 49ers': 'SF', 'New England Patriots': 'NE',
    'Minnesota Vikings': 'MIN', 'Denver Broncos': 'DEN', 'Los Angeles Rams': 'LA', 'New Orleans Saints': 'NO',
    'Detroit Lions': 'DET', 'Dallas Cowboys': 'DAL', 'New York Giants': 'NYG',
    'Buffalo Bills': 'BUF', 'Chicago Bears': 'CHI', 'Green Bay Packers': 'GB','Indianapolis Colts': 'IND',
    'Baltimore Ravens': 'BAL', 'New York Jets': 'NYJ', 'Tampa Bay Buccaneers': 'TB', 'Tennessee Titans': 'TEN',
    'Los Angeles Chargers': 'LAC','Philadelphia Eagles': 'PHI', 'Seattle Seahawks': 'SEA', 'Carolina Panthers': 'CAR',
    'Atlanta Falcons': 'ATL', 'Cincinnati Bengals': 'CIN', 'Houston Texans': 'HOU', 'Kansas City Chiefs': 'KC',
    'Arizona Cardinals': 'ARI', 'Washington Redskins': 'WAS', 'Washington Football Team' : 'WAS',
    'Cleveland Browns': 'CLE',
    'Oakland Raiders': 'LV', 'Las Vegas Raiders' : 'LV',
    'Jacksonville Jaguars': 'JAX',
    'Miami Dolphins': 'MIA'
}
print(len(team_replacements.keys()))
defense_grades_df['Team'] = defense_grades_df['Team'].replace(team_replacements)
defense_grades_df.rename(columns={'Team' : 'DefenseTeam'}, inplace=True)

34


In [167]:
defense_grades_df['current_defence_rank'] = defense_grades_df.groupby('DefenseTeam')['Previous_DEF_rank'].shift(-1)
defense_grades_df.isnull().sum()

DefenseTeam              0
season                   0
Previous_DEF_rank        0
current_defence_rank    32
dtype: int64

Merging with PBP so every play column 'Season Defence Rank'

In [168]:
pbp_exp4 = pd.read_csv('../../Data/play-by-play/pbp_exp5.csv') #features we are using for next experiment

In [169]:
print(pbp_exp4.shape)
pbp_exp8 = pd.merge(pbp_exp4, defense_grades_df, on=['DefenseTeam', 'season'], how='left')
pbp_exp8['current_defence_rank'] = pbp_exp8['current_defence_rank'].astype(int)
print(pbp_exp8.shape)
print(pbp_exp8.isnull().sum().sum()) # no nulls and shape stays 
print(pbp_exp8.duplicated().sum()) #no duplicates

(88336, 48)
(88336, 50)
0
0


In [170]:
pbp_exp8['Player-Injured-On-Play'] = pbp_exp8.pop('Player-Injured-On-Play') #move back to last column

In [171]:
pbp_exp8.to_csv('../../Data/play-by-play/pbp_exp8.csv', index=False)

In [172]:
# defense_grades_df.to_csv('../../Data/pff-data/defense_rankings.csv', index=False)